In [1]:
#Loading necessary libraries for data analysis and machine learning
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from ydata_profiling import ProfileReport

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


## Data Understanding

In [2]:
#Loading the dataset
cervical_data = pd.read_excel('Cervical Cancer Datasets_.xlsx')
cost_data = pd.read_excel('Resources Inventory Cost Sheet.xlsx')
Treatment_data = pd.read_excel('Treatment Costs Sheet.xlsx')


In [3]:
#checking the first few rows of the dataset
cervical_data.head()

,Patient ID,Age,Sexual Partners,First Sexual Activity Age,HPV Test Result,Pap Smear Result,Smoking Status,STDs History,Region,Insrance Covered,Screening Type Last,Recommended Action,Unnamed: 12
0,P0001,18,4,15,NEGATIVE,N,N,Y,Pumwani,Y,PAP SMEAR,REPEAT PAP SMEAR IN 3 YEARS AND FOR HPV VACCINE,NaN
1,P0002,15,1,14,POSITIVE,N,Y,Y,Kakamega,N,HPV DNA,FOR HPV VACCINE AND SEXUAL EDUCATION,NaN
2,P0003,34,1,9,POSITIVE,N,N,Y,Machakos,N,HPV DNA,FOR HPV VACCINE AND SEXUAL EDUCATION,NaN
3,P0004,52,5,16,POSITIVE,N,Y,N,Embu,Y,HPV DNA,"FOR HPV VACCINE, LIFESTYLE AND SEXUAL EDUCATION",NaN
4,P0005,46,3,21,POSITIVE,N,N,N,Mombasa,N,HPV DNA,FOR HPV VACCINE AND SEXUAL EDUCATION,NaN


In [11]:
profile = ProfileReport(cervical_data, title="Cervical Cancer Risk Profile")
profile.to_notebook_iframe() 

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 224.42it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
#Dropping the columns with more than 50% missing values
cervical_data = cervical_data.drop(columns=['Unnamed: 12'], axis=1)

- There are no missing values

In [8]:
#checking the first few rows of the cost data
cost_data.head()

,Facility,Region,Category,Item,Cost (KES),Available Stock
0,Pumwani Maternity Hospital,Pumwani,Medications,Ibuprofen 400mg,1935.25,94
1,Kakamega County Referral Hospital,Kakamega,Medications,Combined Oral Contraceptives,4758.50,28
2,Machakos Level 5 Hospital,Machakos,Medications,Paracetamol 500mg,3686.77,86
3,Embu Level 5 Hospital,Embu,Medications,Paracetamol 500mg,3033.43,75
4,Mombasa County Hospital,Mombasa,Medications,Ibuprofen 400mg,864.49,3


In [12]:
#checking the descriptive statistics of the cost data
profile = ProfileReport(cost_data, title="Cervical Cancer Risk Profile")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 51.77it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Exploratory Data Analysis

In [14]:
# EDA of the cervical data
sns.countplot(x='HPV Test Result', data=cervical_data)
plt.title('Distribution of Biopsy Results') 
plt.xlabel('Biopsy Result')
plt.ylabel('Count')
plt.show()

C:\Users\Wanjiru\AppData\Local\Temp\ipykernel_5108\1513559170.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
